https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_12_to_14.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`

ColBERT tries to address the issue that embedding an entire doc into a small vector inevitably loses information. Instead they generate an embedding for every single token in both the question and the docs. 

They then sum up the pairwise maximum similarity score for each token in the question. E.g. question has 10 tokens, docs have 10000 tokens, iterate each of the 10 tokens, and calc the max across 10000. Then sum the 10 similarity scores.

Frankly doesn't sound even slightly usable outside of leaderboard challenges. Real life would be too slow.

In [1]:
import langchain 
langchain.debug = True 

In [2]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Feb 17, 13:47:32] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [3]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.

    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"] if "extract" in page else None

full_document = get_wikipedia_page("Hayao_Miyazaki")

In [5]:
len(full_document)

68505

In [6]:
RAG.index(
    collection=[full_document],
    index_name="Miyazaki-123",
    max_document_length=180,
    split_documents=True,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Feb 17, 13:50:32] #> Creating directory .ragatouille/colbert/indexes/Miyazaki-123 


[Feb 17, 13:50:34] [0] 		 #> Encoding 122 passages..


/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/4 [00:00<?, ?it/s]/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not avail

[Feb 17, 13:50:51] [0] 		 avg_doclen_est = 131.9262237548828 	 len(local_sample) = 122
[Feb 17, 13:50:51] [0] 		 Creating 1,024 partitions.
[Feb 17, 13:50:51] [0] 		 *Estimated* 16,094 embeddings.
[Feb 17, 13:50:51] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Miyazaki-123/plan.json ..


used 20 iterations (3.2597s) to cluster 15291 items into 1024 clusters
[0.036, 0.041, 0.039, 0.036, 0.033, 0.036, 0.036, 0.036, 0.033, 0.032, 0.033, 0.035, 0.037, 0.037, 0.036, 0.037, 0.032, 0.032, 0.034, 0.037, 0.035, 0.034, 0.034, 0.038, 0.037, 0.032, 0.037, 0.035, 0.034, 0.036, 0.034, 0.036, 0.039, 0.033, 0.035, 0.033, 0.034, 0.034, 0.035, 0.04, 0.034, 0.037, 0.032, 0.033, 0.037, 0.035, 0.034, 0.036, 0.035, 0.033, 0.034, 0.035, 0.034, 0.037, 0.035, 0.035, 0.038, 0.038, 0.04, 0.032, 0.034, 0.037, 0.035, 0.034, 0.037, 0.036, 0.036, 0.037, 0.034, 0.032, 0.037, 0.034, 0.034, 0.034, 0.035, 0.035, 0.035, 0.039, 0.034, 0.035, 0.037, 0.036, 0.033, 0.037, 0.034, 0.035, 0.038, 0.036, 0.033, 0.041, 0.033, 0.036, 0.036, 0.038, 0.034, 0.035, 0.039, 0.035, 0.037, 0.036, 0.038, 0.041, 0.037, 0.035, 0.037, 0.034, 0.035, 0.033, 0.034, 0.032, 0.036, 0.036, 0.034, 0.032, 0.036, 0.036, 0.035, 0.035, 0.038, 0.038, 0.033, 0.033, 0.034, 0.038, 0.033, 0.037, 0.038, 0.034]


0it [00:00, ?it/s]

...

100%|██████████| 4/4 [00:18<00:00,  4.72s/it]
1it [00:19, 19.06s/it]
100%|██████████| 1/1 [00:00<00:00, 1381.07it/s]

[Feb 17, 13:51:14] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 17, 13:51:14] #> Building the emb2pid mapping..
[Feb 17, 13:51:14] len(emb2pid) = 16095



100%|██████████| 1024/1024 [00:00<00:00, 60599.19it/s]

[Feb 17, 13:51:14] #> Saved optimized IVF to .ragatouille/colbert/indexes/Miyazaki-123/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/Miyazaki-123'

In [7]:
# Use as langchain retriever
retriever = RAG.as_langchain_retriever(k=3)
retriever.invoke("What animation studio did Miyazaki found?")

Loading searcher for index Miyazaki-123 for the first time... This may take a few seconds
[Feb 17, 13:52:26] #> Loading codec...
[Feb 17, 13:52:26] #> Loading IVF...
[Feb 17, 13:52:26] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


[Feb 17, 13:52:41] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2626.36it/s]

[Feb 17, 13:52:41] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 160.16it/s]

[Feb 17, 13:52:41] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Feb 17, 13:52:57] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: What animation studio did Miyazaki found?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  7284,  2996,  2106,  2771,  3148, 18637,  2179,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[Document(metadata={}, page_content='=== Studio Ghibli ===\n\n\n==== Early films (1985–1995) ====\nFollowing the success of Nausicaä of the Valley of the Wind, Miyazaki and Takahata founded the animation production company Studio Ghibli on June 15, 1985, as a subsidiary of Tokuma Shoten, with offices in Kichijōji designed by Miyazaki. The studio\'s name had been registered a year earlier; Miyazaki named it after the nickname of the Caproni Ca.309 aircraft, meaning "a hot wind that blows in the desert" in Italian.'),
 Document(metadata={}, page_content='Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. He co-founded Studio Ghibli and serves as its honorary chairman. Over the course of his career, Miyazaki has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of an